In [1]:
#Notebook description

#This notebook is being used to evaluate the techinical market conditions of a single asset and assess
#the appropriate strategy to take in order to maximize returns.

In [2]:
#take all compuation functions and put them in a separate file

#simplify the date x axis on the percent drawdown chart

#default the zoom range to a comfortable range, and create a dropdown to select the time range for Volatility section

#properly label and annoate the garch models

#Remove the VIX charting, its redudnant now that we have the volatility models

In [3]:
#Load libraries 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True
logger.propagate = False
# Load libraries
import sys
sys.path.append(r"e:\Coding Projects\Investment Analysis")
from Quantapp.Plotter import Plotter
from Quantapp.Computation import Computation, Helper
from Quantapp.EconomicData import EconomicData

import numpy as np
import json
import os
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import coint
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor
from plotly.subplots import make_subplots
from datetime import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

#shut down warnings
import warnings
warnings.filterwarnings("ignore")


qc = Computation()
qp = Plotter()
qe = EconomicData()
helper = Helper()


In [4]:
#PARAMETERS

time_frame_week = 7
time_frame_short = 21
time_frame_mid   = 50
time_frame_long = 200

interval = '1d'
period     = '10y'
risk_free_rate = 0.02 / 252  # Annualized risk-free rate divided by trading days

#should be a string or None
ticker_str = 'UNH'



In [5]:
#Load data: underlying data
print(f"Loading data for {ticker_str} with period {period} and interval {interval}")
ticker = yf.Ticker(ticker_str).history(period=period, interval=interval)
ticker = helper.simplify_datetime_index(ticker)

Loading data for UNH with period 10y and interval 1d


In [6]:
#plot geometric brownian motion monte carlo simulation
import yfinance as yf
import numpy as np
import plotly.graph_objects as go

# --- Parameters ---
ticker_str = ticker_str       # Stock ticker
interval = "1d"           # Daily prices
period = "3mo"            # Historical data period
lookback_days = 30        # Days to estimate volatility
projection_days = 30      # Days to simulate into the future
num_paths = 500         # Monte Carlo paths

# --- Function to calculate GBM Monte Carlo paths ---
def calculate_gbm_paths(prices, projection_days=30, lookback_days=30, num_paths=500, mu=None):
    recent_prices = prices[-lookback_days:]
    log_returns = np.log(recent_prices[1:] / recent_prices[:-1])
    sigma = log_returns.std()
    if mu is None:
        mu = 0

    S0 = prices[-1]
    T = projection_days
    N = num_paths
    dt = 1/252

    price_paths = np.zeros((T, N))
    price_paths[0, :] = S0

    for t in range(1, T):
        z = np.random.standard_normal(N)
        price_paths[t, :] = price_paths[t-1, :] * np.exp((mu - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z)

    median_path = np.median(price_paths, axis=1)
    p5 = np.percentile(price_paths, 5, axis=1)
    p95 = np.percentile(price_paths, 95, axis=1)

    return price_paths, median_path, p5, p95

# --- Function to plot historical + Monte Carlo paths ---
def plot_gbm_paths(dates, prices, price_paths, median_path, p5, p95, ticker_str="Ticker"):
    fig = go.Figure()

    # Historical price
    fig.add_trace(go.Scatter(
        x=dates,
        y=prices,
        mode='lines',
        name='Historical Price',
        line=dict(color='black', width=2)
    ))

    # Monte Carlo paths (faint)
    T, N = price_paths.shape
    future_dates = [dates[-1] + np.timedelta64(i+1, 'D') for i in range(T)]
    for i in range(N):
        fig.add_trace(go.Scatter(
            x=future_dates,
            y=price_paths[:, i],
            mode='lines',
            line=dict(color='blue', width=1),
            opacity=0.05,
            showlegend=False
        ))

    # 5–95% confidence cloud
    fig.add_trace(go.Scatter(
        x=future_dates + future_dates[::-1],
        y=np.concatenate([p95, p5[::-1]]),
        fill='toself',
        fillcolor='rgba(0,100,255,0.25)',
        line=dict(color='rgba(0,0,255,0)'),
        hoverinfo='skip',
        showlegend=True,
        name='5–95% Range'
    ))

    # Median path
    fig.add_trace(go.Scatter(
        x=future_dates,
        y=median_path,
        mode='lines',
        line=dict(color='red', width=2),
        name='Median Path'
    ))

    # Horizontal lines and labels for terminal percentiles
    terminal_prices = np.percentile(price_paths[-1, :], [5, 25, 50, 75, 95])
    terminal_names = ['5%', '25%', '50% (Median)', '75%', '95%']
    for price, name in zip(terminal_prices, terminal_names):
        fig.add_trace(go.Scatter(
            x=[dates[-1], future_dates[-1]],
            y=[price, price],
            mode='lines',
            line=dict(dash='dash', width=2),
            name=f'Terminal {name}'
        ))
        fig.add_annotation(
            x=future_dates[-1],
            y=price,
            xanchor='left',
            text=f"{name}: {price:.2f}",
            showarrow=False,
            font=dict(color='black')
        )

    fig.update_layout(
        title=f"Historical Price + 30-Day Monte Carlo Simulation ({ticker_str})",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_white",
        width=1200,
        height=800
    )

    fig.show()


# --- Main execution ---

# Load historical data
data = yf.Ticker(ticker_str).history(period=period, interval=interval)
prices = data['Close'].values
dates = data.index

# Calculate Monte Carlo paths
price_paths, median_path, p5, p95 = calculate_gbm_paths(
    prices,
    projection_days=projection_days,
    lookback_days=lookback_days,
    num_paths=num_paths
)

# Plot results
plot_gbm_paths(dates, prices, price_paths, median_path, p5, p95, ticker_str=ticker_str)


In [7]:
#Load data: Expiration Dates 
print(f"Loading options expiration dates for {ticker_str}")
#retreive options data from yahoo finance
options_expiration_dates = yf.Ticker(ticker_str).options
options_expiration_dates = pd.DataFrame(options_expiration_dates, columns=['Expiration Date'])

#add another column called date till expiration
options_expiration_dates['Date Till Expiration'] = pd.to_datetime(options_expiration_dates['Expiration Date']) - pd.to_datetime('today')
options_expiration_dates['Date Till Expiration'] = options_expiration_dates['Date Till Expiration'].dt.days

Loading options expiration dates for UNH


In [8]:
#retreive the entire options chain 
print(f"Loading options chain for {ticker_str}")
spot_price = yf.Ticker(ticker_str).history(period='1d', interval='1d').iloc[-1]['Close']
call_contracts = yf.Ticker(ticker_str).option_chain(options_expiration_dates['Expiration Date'][0]).calls 
#retreive ever option contract for every expiration date and save each contract chain within its own value in a dictionary just for calls
call_contract_chain = {}
put_contract_chain = {}

for expiration_date in options_expiration_dates['Expiration Date']:
    drop_columns = [ 'lastTradeDate','contractSize','currency','percentChange','change']
    add_collumns = ['Days Till Expiration','Expiration Date']
    
    
    call_contract_chain[expiration_date] = yf.Ticker(ticker_str).option_chain(expiration_date).calls
    call_contract_chain[expiration_date] = call_contract_chain[expiration_date].drop(columns=drop_columns)
    call_contract_chain[expiration_date]['Days Till Expiration'] = (pd.to_datetime(expiration_date) - pd.to_datetime('today')).days
    call_contract_chain[expiration_date]['Expiration Date'] = expiration_date
    call_contract_chain[expiration_date]['bid-ask spread'] = call_contract_chain[expiration_date]['ask'] - call_contract_chain[expiration_date]['bid']
    call_contract_chain[expiration_date]['Expiration day'] = pd.to_datetime(expiration_date).day
    call_contract_chain[expiration_date]['Expiration day name'] = pd.to_datetime(expiration_date).strftime('%A')  # Get the day of the week as a string, e.g., 'Monday', 'Tuesday', etc.
    call_contract_chain[expiration_date]['Type'] = 'Call'
    call_contract_chain[expiration_date]['mid'] = (call_contract_chain[expiration_date]['bid'] + call_contract_chain[expiration_date]['ask']) / 2
    
    put_contract_chain[expiration_date] = yf.Ticker(ticker_str).option_chain(expiration_date).puts
    put_contract_chain[expiration_date] = put_contract_chain[expiration_date].drop(columns=drop_columns)
    put_contract_chain[expiration_date]['Days Till Expiration'] = (pd.to_datetime(expiration_date) - pd.to_datetime('today')).days
    put_contract_chain[expiration_date]['Expiration Date'] = expiration_date
    put_contract_chain[expiration_date]['bid-ask spread'] = put_contract_chain[expiration_date]['ask'] - put_contract_chain[expiration_date]['bid']
    put_contract_chain[expiration_date]['Expiration day'] = pd.to_datetime(expiration_date).day
    put_contract_chain[expiration_date]['Expiration day name'] = pd.to_datetime(expiration_date).strftime('%A')  # Get the day of the week as a string, e.g., 'Monday', 'Tuesday', etc.
    put_contract_chain[expiration_date]['Type'] = 'Put'
    put_contract_chain[expiration_date]['mid'] = (put_contract_chain[expiration_date]['bid'] + put_contract_chain[expiration_date]['ask']) / 2

#all contract series concatenated into a single dataframe
call_contract_chain_concat = pd.concat(call_contract_chain.values(), ignore_index=True)
put_contract_chain_concat = pd.concat(put_contract_chain.values(), ignore_index=True)
all_contracts_concat = pd.concat([call_contract_chain_concat, put_contract_chain_concat], ignore_index=True)

all_contracts_concat 

Loading options chain for UNH


,contractSymbol,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,inTheMoney,Days Till Expiration,Expiration Date,bid-ask spread,Expiration day,Expiration day name,Type,mid
0,UNH250919C00130000,130.0,203.85,206.50,207.90,1.0,1.0,10.304691,True,-1,2025-09-19,1.40,19,Friday,Call,207.200
1,UNH250919C00135000,135.0,219.55,201.55,202.85,65.0,0.0,9.929691,True,-1,2025-09-19,1.30,19,Friday,Call,202.200
2,UNH250919C00140000,140.0,198.25,196.50,197.50,1.0,5.0,9.355473,True,-1,2025-09-19,1.00,19,Friday,Call,197.000
3,UNH250919C00150000,150.0,190.71,185.95,187.65,2.0,2.0,8.468755,True,-1,2025-09-19,1.70,19,Friday,Call,186.800
4,UNH250919C00155000,155.0,185.72,181.60,182.90,2.0,0.0,8.613286,True,-1,2025-09-19,1.30,19,Friday,Call,182.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271,UNH271217P00450000,450.0,135.00,130.90,135.90,2.0,93.0,0.271293,True,818,2027-12-17,5.00,17,Friday,Put,133.400
2272,UNH271217P00460000,460.0,132.10,136.95,142.10,1.0,254.0,0.258842,True,818,2027-12-17,5.15,17,Friday,Put,139.525
2273,UNH271217P00470000,470.0,140.00,144.55,150.25,1.0,7.0,0.256676,True,818,2027-12-17,5.70,17,Friday,Put,147.400
2274,UNH271217P00490000,490.0,160.60,162.45,166.20,NaN,NaN,0.246757,True,818,2027-12-17,3.75,17,Friday,Put,164.325


In [9]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from scipy.stats import norm

# --- Setup ticker and options chains ---

ticker = yf.Ticker(ticker_str)
options_expiration_dates = pd.DataFrame(ticker.options, columns=['Expiration Date'])

# Retrieve options chains
call_contract_chain = {}
put_contract_chain = {}

for expiration_date in options_expiration_dates['Expiration Date']:
    calls = ticker.option_chain(expiration_date).calls.drop(columns=['lastTradeDate','contractSize','currency','percentChange','change'])
    puts = ticker.option_chain(expiration_date).puts.drop(columns=['lastTradeDate','contractSize','currency','percentChange','change'])
    
    for df in [calls, puts]:
        df['Days Till Expiration'] = (pd.to_datetime(expiration_date) - pd.to_datetime('today')).days
        df['Expiration Date'] = expiration_date
        df['Type'] = 'Call' if df is calls else 'Put'
        df['mid'] = (df['bid'] + df['ask']) / 2
    
    call_contract_chain[expiration_date] = calls
    put_contract_chain[expiration_date] = puts

# --- Spot and historical vol ---
spot_price = ticker.history(period='1y')['Close'].iloc[-1]
price_series = ticker.history(period='1y')['Close']
log_returns = np.log(price_series / price_series.shift(1)).dropna()
annualized_vol = log_returns.std() * np.sqrt(252)

# --- Function to compute strike for a given PoT ---
def strike_for_pot(spot, sigma, T, prob=0.15, option_type='call'):
    z = norm.ppf(1 - prob)
    if option_type == 'call':
        return spot * np.exp(sigma * np.sqrt(T) * z)
    else:
        return spot * np.exp(-sigma * np.sqrt(T) * z)

# --- Initialize figure ---
expirations = sorted(call_contract_chain.keys())
fig = go.Figure()

# Add traces for first expiration
first_exp = expirations[0]
fig.add_trace(go.Bar(
    x=call_contract_chain[first_exp]['strike'],
    y=call_contract_chain[first_exp]['openInterest'],
    name='Calls',
    marker_color='blue'
))
fig.add_trace(go.Bar(
    x=put_contract_chain[first_exp]['strike'],
    y=put_contract_chain[first_exp]['openInterest'],
    name='Puts',
    marker_color='green'
))

# Max OI strikes
max_oi_call_strike = call_contract_chain[first_exp].loc[call_contract_chain[first_exp]['openInterest'].idxmax(), 'strike']
max_oi_put_strike = put_contract_chain[first_exp].loc[put_contract_chain[first_exp]['openInterest'].idxmax(), 'strike']

T = call_contract_chain[first_exp]['Days Till Expiration'].iloc[0] / 252

# Add spot & max OI lines
fig.add_vline(x=spot_price, line_color='red', line_dash='dash', line_width=2, annotation_text='Spot', annotation_position='top left')
fig.add_vline(x=max_oi_call_strike, line_color='blue', line_dash='dot', line_width=2, annotation_text='Max Call OI', annotation_position='bottom left')
fig.add_vline(x=max_oi_put_strike, line_color='green', line_dash='dot', line_width=2, annotation_text='Max Put OI', annotation_position='bottom left')

# --- 15% and 30% PoT ranges ---
call_lower_15 = strike_for_pot(spot_price, annualized_vol, T, prob=0.15, option_type='call')
put_upper_15 = strike_for_pot(spot_price, annualized_vol, T, prob=0.15, option_type='put')

call_lower_30 = strike_for_pot(spot_price, annualized_vol, T, prob=0.30, option_type='call')
put_upper_30 = strike_for_pot(spot_price, annualized_vol, T, prob=0.30, option_type='put')

# Highlight 15% PoT band (orange)
fig.add_shape(
    type="rect",
    x0=put_upper_15, x1=call_lower_15,
    y0=0, y1=max(call_contract_chain[first_exp]['openInterest'].max(), put_contract_chain[first_exp]['openInterest'].max())*1.05,
    fillcolor="orange",
    opacity=0.2,
    layer="below",
    line_width=0,
)

# Highlight 30% PoT band (blue)
fig.add_shape(
    type="rect",
    x0=put_upper_30, x1=call_lower_30,
    y0=0, y1=max(call_contract_chain[first_exp]['openInterest'].max(), put_contract_chain[first_exp]['openInterest'].max())*1.05,
    fillcolor="blue",
    opacity=0.2,
    layer="below",
    line_width=0,
)

# --- Dropdown for expirations ---
buttons = []
for exp in expirations:
    visible = [False] * (len(expirations)*2)
    visible[expirations.index(exp)*2] = True
    visible[expirations.index(exp)*2+1] = True
    
    T = call_contract_chain[exp]['Days Till Expiration'].iloc[0] / 252
    max_call = call_contract_chain[exp].loc[call_contract_chain[exp]['openInterest'].idxmax(), 'strike']
    max_put = put_contract_chain[exp].loc[put_contract_chain[exp]['openInterest'].idxmax(), 'strike']
    
    # 15% and 30% PoT
    call_lower_15 = strike_for_pot(spot_price, annualized_vol, T, prob=0.15, option_type='call')
    put_upper_15 = strike_for_pot(spot_price, annualized_vol, T, prob=0.15, option_type='put')
    call_lower_30 = strike_for_pot(spot_price, annualized_vol, T, prob=0.30, option_type='call')
    put_upper_30 = strike_for_pot(spot_price, annualized_vol, T, prob=0.30, option_type='put')
    
    shapes = [
        dict(type='line', x0=spot_price, x1=spot_price, y0=0, y1=max(call_contract_chain[exp]['openInterest'].max(),
                                                                     put_contract_chain[exp]['openInterest'].max())*1.05,
             line=dict(color='red', dash='dash')),
        dict(type='line', x0=max_call, x1=max_call, y0=0, y1=call_contract_chain[exp]['openInterest'].max()*1.05,
             line=dict(color='blue', dash='dot')),
        dict(type='line', x0=max_put, x1=max_put, y0=0, y1=put_contract_chain[exp]['openInterest'].max()*1.05,
             line=dict(color='green', dash='dot')),
        dict(type="rect", x0=put_upper_15, x1=call_lower_15,
             y0=0, y1=max(call_contract_chain[exp]['openInterest'].max(), put_contract_chain[exp]['openInterest'].max())*1.05,
             fillcolor="orange", opacity=0.2, layer="below", line_width=0),
        dict(type="rect", x0=put_upper_30, x1=call_lower_30,
             y0=0, y1=max(call_contract_chain[exp]['openInterest'].max(), put_contract_chain[exp]['openInterest'].max())*1.05,
             fillcolor="blue", opacity=0.2, layer="below", line_width=0)
    ]
    
    button = dict(
        label=exp,
        method='update',
        args=[{'visible': visible},
              {'title': f'Open Interest & PoT Ranges: {exp}', 'shapes': shapes}]
    )
    buttons.append(button)

# Add remaining traces (hidden)
for exp in expirations[1:]:
    fig.add_trace(go.Bar(x=call_contract_chain[exp]['strike'], y=call_contract_chain[exp]['openInterest'], name='Calls', marker_color='blue', visible=False))
    fig.add_trace(go.Bar(x=put_contract_chain[exp]['strike'], y=put_contract_chain[exp]['openInterest'], name='Puts', marker_color='green', visible=False))

# Layout
fig.update_layout(
    updatemenus=[dict(active=0, buttons=buttons, x=0, y=1.15, xanchor='left', yanchor='top')],
    barmode='group',
    xaxis_title='Strike Price',
    yaxis_title='Open Interest',
    template='plotly_white',
    height=600
)

fig.show()


In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Compute totals
total_oi_calls = [call_contract_chain[exp]['openInterest'].sum() for exp in expirations]
total_oi_puts = [put_contract_chain[exp]['openInterest'].sum() for exp in expirations]
dte_list = [call_contract_chain[exp]['Days Till Expiration'].iloc[0] for exp in expirations]

# Aggregate total OI (calls + puts)
total_oi_all = [c + p for c, p in zip(total_oi_calls, total_oi_puts)]

# Convert expirations to string for categorical x-axis
exp_labels = [str(exp) for exp in expirations]

# Create subplots (2 rows)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.15,
                    subplot_titles=('Total Call vs Put Open Interest', 'Aggregate Open Interest (Calls + Puts)'))

# --- Top plot: Calls vs Puts ---
fig.add_trace(go.Bar(
    x=exp_labels,
    y=total_oi_calls,
    name='Calls',
    marker_color='blue',
    hovertemplate='Expiration: %{x}<br>Total Call OI: %{y}<br>DTE: %{customdata}',
    customdata=dte_list
), row=1, col=1)

fig.add_trace(go.Bar(
    x=exp_labels,
    y=total_oi_puts,
    name='Puts',
    marker_color='green',
    hovertemplate='Expiration: %{x}<br>Total Put OI: %{y}<br>DTE: %{customdata}',
    customdata=dte_list
), row=1, col=1)

# --- Bottom plot: Aggregate OI ---
fig.add_trace(go.Bar(
    x=exp_labels,
    y=total_oi_all,
    name='Calls + Puts',
    marker_color='orange',
    hovertemplate='Expiration: %{x}<br>Aggregate OI: %{y}<br>DTE: %{customdata}',
    customdata=dte_list
), row=2, col=1)

# Layout
fig.update_layout(
    title='Options Open Interest Overview',
    xaxis_title='Option Expiration',
    yaxis_title='Open Interest',
    barmode='group',
    template='plotly_white',
    height=800
)

# Bottom x-axis title
fig.update_xaxes(title_text='Option Expiration', row=2, col=1)

# Force categorical x-axis to remove gaps
fig.update_xaxes(type='category', row=1, col=1)
fig.update_xaxes(type='category', row=2, col=1)

fig.show()


In [11]:
#plot the ATM IV and Realized Volatility
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Define ATM IV fetcher
def get_atm_iv_for_expiration(expiration_date, contract_chain):
    contracts = contract_chain[expiration_date]
    idx = (contracts['strike'] - spot_price).abs().idxmin()
    return contracts.loc[idx, 'impliedVolatility']

# Get ATM IVs
atm_iv_call = pd.DataFrame({
    'Expiration Date': list(call_contract_chain.keys()),
    'ATM IV Call': [get_atm_iv_for_expiration(exp, call_contract_chain) for exp in call_contract_chain]
})
atm_iv_put = pd.DataFrame({
    'Expiration Date': list(put_contract_chain.keys()),
    'ATM IV Put': [get_atm_iv_for_expiration(exp, put_contract_chain) for exp in put_contract_chain]
})

# Calculate realized vol for each expiration
def get_realized_vol_for_expiration(expiration_date):
    days = (pd.to_datetime(expiration_date) - pd.to_datetime('today')).days
    if 1 < days < len(ticker):
        returns = np.log(ticker['Close'] / ticker['Close'].shift(1)).dropna()
        window_returns = returns[-days:]
        realized_vol = window_returns.std() * np.sqrt(252)
        return realized_vol
    return np.nan

# Apply realized vol
atm_iv_call['Realized Vol'] = atm_iv_call['Expiration Date'].apply(get_realized_vol_for_expiration)
atm_iv_put['Realized Vol'] = atm_iv_call['Realized Vol']

# Compute Days Till Expiration
atm_iv_call['Days Till Expiration'] = atm_iv_call['Expiration Date'].apply(
    lambda d: (pd.to_datetime(d) - pd.to_datetime('today')).days
)
atm_iv_put['Days Till Expiration'] = atm_iv_call['Days Till Expiration']

# Merge
atm_df = atm_iv_call.merge(atm_iv_put, on=['Expiration Date', 'Realized Vol', 'Days Till Expiration'], how='outer')

# Compute spreads
atm_df['IV-RV Call'] = atm_df['ATM IV Call'] - atm_df['Realized Vol']
atm_df['IV-RV Put'] = atm_df['ATM IV Put'] - atm_df['Realized Vol']

# Calculate Put-Call IV Skew
atm_df['IV Skew'] = atm_df['ATM IV Put'] - atm_df['ATM IV Call']

# Plot
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    subplot_titles=(
        'ATM IV and Realized Volatility',
        'IV - Realized Vol Spread',
        'Put-Call IV Skew'
    )
)

# First subplot: ATM IV and realized vol
fig.add_trace(go.Scatter(
    x=atm_df['Days Till Expiration'], y=atm_df['ATM IV Call'],
    mode='lines+markers', name='ATM IV Call', line=dict(color='blue')
), row=1, col=1)
fig.add_trace(go.Scatter(
    x=atm_df['Days Till Expiration'], y=atm_df['ATM IV Put'],
    mode='lines+markers', name='ATM IV Put', line=dict(color='green')
), row=1, col=1)
fig.add_trace(go.Scatter(
    x=atm_df['Days Till Expiration'], y=atm_df['Realized Vol'],
    mode='lines+markers', name='Realized Volatility', line=dict(color='red')
), row=1, col=1)

# Second subplot: IV - Realized Vol spreads
fig.add_trace(go.Scatter(
    x=atm_df['Days Till Expiration'], y=atm_df['IV-RV Call'],
    mode='lines+markers', name='IV-RV (Call)', line=dict(color='blue', dash='dash')
), row=2, col=1)
fig.add_trace(go.Scatter(
    x=atm_df['Days Till Expiration'], y=atm_df['IV-RV Put'],
    mode='lines+markers', name='IV-RV (Put)', line=dict(color='green', dash='dash')
), row=2, col=1)
fig.add_hline(y=0, line_color='black', line_dash='dash', row=2, col=1)

# Third subplot: Put-Call IV Skew
fig.add_trace(go.Scatter(
    x=atm_df['Days Till Expiration'], y=atm_df['IV Skew'],
    mode='lines+markers', name='Put-Call IV Skew', line=dict(color='purple')
), row=3, col=1)

fig.update_layout(
    title=f'Volatility & IV Skew Analysis - {ticker_str}',
    xaxis_title='Days Till Expiration',
    yaxis_title='Volatility',
    height=1200,
    showlegend=True
)

fig.show()


TypeError: object of type 'Ticker' has no len()

In [12]:
#Plot IV - Realized Volatility by Strike for Calls and Puts
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Historical price data (must be a Series indexed by date, most recent last)
price_series = ticker['Close']
log_returns = np.log(price_series / price_series.shift(1))

# Spot price for reference line
spot_price = price_series.iloc[-1]

# Create subplot figure with 1 row, 2 columns (calls on left, puts on right)
fig = make_subplots(rows=1, cols=2, subplot_titles=("Calls: IV - Realized Vol", "Puts: IV - Realized Vol"))

# Sort expiration dates
call_expirations = sorted(call_contract_chain.keys())
put_expirations = sorted(put_contract_chain.keys())

today = pd.to_datetime('today')

# Plot calls in first subplot (col=1)
for exp in call_expirations:
    df = call_contract_chain[exp].copy()
    df_sorted = df.sort_values('strike')

    exp_date = pd.to_datetime(exp)
    days_till_exp = (exp_date - today).days

    if days_till_exp < 2 or days_till_exp > len(log_returns):
        continue

    realized_vol_n = log_returns.rolling(window=days_till_exp).std().iloc[-1] * np.sqrt(252)
    df_sorted['iv_minus_realized'] = df_sorted['impliedVolatility'] - realized_vol_n

    fig.add_trace(go.Scatter(
        x=df_sorted['strike'],
        y=df_sorted['iv_minus_realized'],
        mode='lines+markers',
        name=f"{days_till_exp}d — Exp: {exp_date.strftime('%b %d, %Y')}",
        legendgroup='Calls',
        showlegend=True if exp == call_expirations[0] else False  # show legend only once
    ), row=1, col=1)

# Plot puts in second subplot (col=2)
for exp in put_expirations:
    df = put_contract_chain[exp].copy()
    df_sorted = df.sort_values('strike')

    exp_date = pd.to_datetime(exp)
    days_till_exp = (exp_date - today).days

    if days_till_exp < 2 or days_till_exp > len(log_returns):
        continue

    realized_vol_n = log_returns.rolling(window=days_till_exp).std().iloc[-1] * np.sqrt(252)
    df_sorted['iv_minus_realized'] = df_sorted['impliedVolatility'] - realized_vol_n

    fig.add_trace(go.Scatter(
        x=df_sorted['strike'],
        y=df_sorted['iv_minus_realized'],
        mode='lines+markers',
        name=f"{days_till_exp}d — Exp: {exp_date.strftime('%b %d, %Y')}",
        legendgroup='Puts',
        showlegend=True if exp == put_expirations[0] else False  # show legend only once
    ), row=1, col=2)

# Add horizontal zero line to both plots
fig.add_hline(y=0, line_color='black', line_dash='dash', row=1, col=1)
fig.add_hline(y=0, line_color='black', line_dash='dash', row=1, col=2)

# Add vertical line for spot price in both plots
fig.add_vline(x=spot_price, line_color='red', line_dash='dash', annotation_text='Spot Price', annotation_position='top left', row=1, col=1)
fig.add_vline(x=spot_price, line_color='red', line_dash='dash', annotation_text='Spot Price', annotation_position='top left', row=1, col=2)

# Update layout
fig.update_layout(
    title='IV - Realized Volatility by Strike for Calls and Puts',
    height=1000,
    template='plotly_white',
    xaxis=dict(
        type='linear',
        title='Strike Price (Calls)'
    ),
    xaxis2=dict(
        type='linear',
        title='Strike Price (Puts)'
    )
)

fig.show()


TypeError: 'Ticker' object is not subscriptable

In [13]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

today = pd.to_datetime('today')

# --- Helper function to calculate median IV - Realized Vol ---
def calc_median_iv_minus_realized(contract_chain, filter_func=None):
    median_dict = {}
    for exp in contract_chain.keys():
        df = contract_chain[exp].copy()
        if filter_func:
            df = df[filter_func(df)]
        if df.empty:
            continue
        exp_date = pd.to_datetime(exp)
        days_till_exp = (exp_date - today).days
        if days_till_exp < 2 or days_till_exp > len(log_returns):
            continue
        realized_vol_n = log_returns.rolling(window=days_till_exp).std().iloc[-1] * np.sqrt(252)
        median_val = (df['impliedVolatility'] - realized_vol_n).median()
        median_dict[exp_date] = median_val
    return median_dict

# --- Top subplot: All strikes ---
median_calls_all = calc_median_iv_minus_realized(call_contract_chain)
median_puts_all = calc_median_iv_minus_realized(put_contract_chain)

# --- Bottom subplot: OTM strikes ---
median_calls_otm = calc_median_iv_minus_realized(
    call_contract_chain,
    filter_func=lambda df: df['strike'] > spot_price
)
median_puts_otm = calc_median_iv_minus_realized(
    put_contract_chain,
    filter_func=lambda df: df['strike'] < spot_price
)

# --- Create subplots ---
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.15,
    subplot_titles=("Median IV minus Realized Vol (All Strikes)", "OTM Median IV minus Realized Vol")
)

# --- Top subplot traces ---
df_calls_all = pd.DataFrame({'Expiration': list(median_calls_all.keys()), 'Median': list(median_calls_all.values()), 'Type': 'Call'})
df_puts_all = pd.DataFrame({'Expiration': list(median_puts_all.keys()), 'Median': list(median_puts_all.values()), 'Type': 'Put'})
df_all = pd.concat([df_calls_all, df_puts_all])
df_all['DTE'] = (df_all['Expiration'] - today).dt.days

# Find max for top subplot
max_idx = df_all['Median'].idxmax()
max_row = df_all.loc[max_idx]

# Calls
call_all_data = df_all[df_all['Type']=='Call']
fig.add_trace(go.Scatter(
    x=call_all_data['Expiration'],
    y=call_all_data['Median'],
    mode='lines+markers',
    name='Calls',
    line=dict(color='blue'),
    hovertemplate='DTE: %{customdata} days<br>Median IV-Realized: %{y:.2%}',
    customdata=call_all_data['DTE']
), row=1, col=1)

# Puts
put_all_data = df_all[df_all['Type']=='Put']
fig.add_trace(go.Scatter(
    x=put_all_data['Expiration'],
    y=put_all_data['Median'],
    mode='lines+markers',
    name='Puts',
    line=dict(color='green'),
    hovertemplate='DTE: %{customdata} days<br>Median IV-Realized: %{y:.2%}',
    customdata=put_all_data['DTE']
), row=1, col=1)

# MAX marker
fig.add_trace(go.Scatter(
    x=[max_row['Expiration']],
    y=[max_row['Median']],
    mode='markers+text',
    text=['MAX'],
    textposition='top center',
    marker=dict(color='red', size=12),
    hovertemplate='DTE: %{customdata} days<br>Median IV-Realized: %{y:.2%}',
    customdata=[max_row['DTE']],
    showlegend=False
), row=1, col=1)

# Horizontal zero line
fig.add_hline(y=0, line_dash='dash', line_color='black', row=1, col=1)

# --- Bottom subplot traces (OTM) ---
df_calls_otm = pd.DataFrame({'Expiration': list(median_calls_otm.keys()), 'Median': list(median_calls_otm.values()), 'Type': 'Call_OTM'})
df_puts_otm = pd.DataFrame({'Expiration': list(median_puts_otm.keys()), 'Median': list(median_puts_otm.values()), 'Type': 'Put_OTM'})
df_otm = pd.concat([df_calls_otm, df_puts_otm])
df_otm['DTE'] = (df_otm['Expiration'] - today).dt.days

# Calls OTM
call_otm_data = df_otm[df_otm['Type']=='Call_OTM']
fig.add_trace(go.Scatter(
    x=call_otm_data['Expiration'],
    y=call_otm_data['Median'],
    mode='lines+markers',
    name='Calls (OTM)',
    line=dict(color='blue'),
    hovertemplate='DTE: %{customdata} days<br>Median IV-Realized: %{y:.2%}',
    customdata=call_otm_data['DTE']
), row=2, col=1)

# Puts OTM
put_otm_data = df_otm[df_otm['Type']=='Put_OTM']
fig.add_trace(go.Scatter(
    x=put_otm_data['Expiration'],
    y=put_otm_data['Median'],
    mode='lines+markers',
    name='Puts (OTM)',
    line=dict(color='green'),
    hovertemplate='DTE: %{customdata} days<br>Median IV-Realized: %{y:.2%}',
    customdata=put_otm_data['DTE']
), row=2, col=1)

# Horizontal zero line for bottom
fig.add_hline(y=0, line_dash='dash', line_color='black', row=2, col=1)

# Layout
fig.update_layout(
    height=1000,
    title='Median IV minus Realized Vol: All Strikes vs OTM Strikes',
    xaxis=dict(type='date'),  # top subplot
    xaxis2=dict(type='date', title='Expiration'),  # bottom subplot
    yaxis=dict(title='Median IV - Realized Vol'),  # top subplot
    yaxis2=dict(title='Median IV - Realized Vol')  # bottom subplot
)


fig.show()


In [14]:
#plot SVI surface for calls and puts
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import minimize
import scipy.interpolate as interp



# Fetch spot price
spot_price = yf.Ticker(ticker_str).history(period='1d')['Close'][0]
print(f"Spot price for {ticker_str}: {spot_price}")

def svi_total_variance(k, a, b, rho, m, sigma):
    return a + b * (rho * (k - m) + np.sqrt((k - m)**2 + sigma**2))

def svi_objective(params, k, total_var):
    a, b, rho, m, sigma = params
    model_var = svi_total_variance(k, a, b, rho, m, sigma)
    return np.sum((model_var - total_var)**2)

def fit_svi(k, total_var):
    x0 = [0.1, 0.1, 0.0, 0.0, 0.1]
    bounds = [(-1, 1), (1e-5, 5), (-0.999, 0.999), (-5, 5), (1e-5, 5)]
    res = minimize(svi_objective, x0, args=(k, total_var), bounds=bounds, method='L-BFGS-B')
    return res.x if res.success else None

def get_realized_vol_surface(ticker, dtes):
    hist = yf.Ticker(ticker).history(period=f'{int(max(dtes) + 30)}d')
    hist['returns'] = np.log(hist['Close'] / hist['Close'].shift(1))
    hist.dropna(inplace=True)

    rv_data = []
    for dte in sorted(set(dtes)):
        dte = int(dte)
        sub_ret = hist['returns'].iloc[-dte:]
        if len(sub_ret) >= dte * 0.8:  # skip if not enough data
            realized_vol = np.std(sub_ret) * np.sqrt(252)
            rv_data.append((dte, realized_vol))
    return pd.DataFrame(rv_data, columns=['Days Till Expiration', 'Realized Vol'])

call_df = call_contract_chain_concat.copy()
call_df = call_df[call_df['impliedVolatility'].notna() & (call_df['impliedVolatility'] > 0)].copy()
call_df['T'] = call_df['Days Till Expiration'] / 365.0

put_df = put_contract_chain_concat.copy()
put_df = put_df[put_df['impliedVolatility'].notna() & (put_df['impliedVolatility'] > 0)].copy()
put_df['T'] = put_df['Days Till Expiration'] / 365.0

def fit_svi_surface(df, spot_price):
    unique_Ts = np.sort(df['T'].unique())
    svi_params_per_T = {}

    for T in unique_Ts:
        slice_df = df[df['T'] == T]
        if len(slice_df) < 5:
            continue
        K = slice_df['strike'].values
        iv = slice_df['impliedVolatility'].values
        total_var = iv**2 * T
        k = np.log(K / spot_price)
        params = fit_svi(k, total_var)
        if params is not None:
            svi_params_per_T[T] = params

    Ts = np.array(sorted(svi_params_per_T.keys()))
    params_array = np.array([svi_params_per_T[T] for T in Ts])
    param_interpolators = [
        interp.interp1d(Ts, params_array[:, i], kind='linear', fill_value='extrapolate')
        for i in range(5)
    ]

    strike_grid = np.linspace(df['strike'].min(), df['strike'].max(), 50)
    T_grid = np.linspace(df['T'].min(), df['T'].max(), 30)
    iv_surface = np.full((len(T_grid), len(strike_grid)), np.nan)

    for i, T in enumerate(T_grid):
        a, b, rho, m, sigma = [f(T) for f in param_interpolators]
        k_vals = np.log(strike_grid / spot_price)
        total_var = svi_total_variance(k_vals, a, b, rho, m, sigma)
        iv_surface[i, :] = np.sqrt(total_var / T)

    return strike_grid, T_grid, iv_surface

call_strike_grid, call_T_grid, call_iv_svi_surface = fit_svi_surface(call_df, spot_price)
put_strike_grid, put_T_grid, put_iv_svi_surface = fit_svi_surface(put_df, spot_price)

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'scene'}, {'type': 'scene'}]],
    subplot_titles=('Call Options IV & SVI Surface', 'Put Options IV & SVI Surface')
)

# Add Call raw IV scatter
fig.add_trace(go.Scatter3d(
    x=call_df['strike'],
    y=call_df['Days Till Expiration'],
    z=call_df['impliedVolatility'],
    mode='markers',
    marker=dict(size=3, color=call_df['impliedVolatility'], colorscale='Viridis', opacity=0.8),
    name='Call Raw IV',
    hovertemplate='Strike: %{x}<br>Days: %{y}<br>IV: %{z:.3f}<extra></extra>'
), row=1, col=1)

# Add Call SVI surface
fig.add_trace(go.Surface(
    x=np.meshgrid(call_strike_grid, call_T_grid * 365)[0],
    y=np.meshgrid(call_strike_grid, call_T_grid * 365)[1],
    z=call_iv_svi_surface,
    colorscale='Viridis',
    opacity=0.7,
    showscale=True,
    colorbar=dict(title='IV'),
    name='Call SVI Surface'
), row=1, col=1)

# Add Put raw IV scatter
fig.add_trace(go.Scatter3d(
    x=put_df['strike'],
    y=put_df['Days Till Expiration'],
    z=put_df['impliedVolatility'],
    mode='markers',
    marker=dict(size=3, color=put_df['impliedVolatility'], colorscale='Cividis', opacity=0.8),
    name='Put Raw IV',
    hovertemplate='Strike: %{x}<br>Days: %{y}<br>IV: %{z:.3f}<extra></extra>'
), row=1, col=2)

# Add Put SVI surface
fig.add_trace(go.Surface(
    x=np.meshgrid(put_strike_grid, put_T_grid * 365)[0],
    y=np.meshgrid(put_strike_grid, put_T_grid * 365)[1],
    z=put_iv_svi_surface,
    colorscale='Cividis',
    opacity=0.7,
    showscale=True,
    colorbar=dict(title='IV'),
    name='Put SVI Surface'
), row=1, col=2)

# Compute consistent axis ranges
strike_min = min(call_df['strike'].min(), put_df['strike'].min())
strike_max = max(call_df['strike'].max(), put_df['strike'].max())
days_min = min(call_df['Days Till Expiration'].min(), put_df['Days Till Expiration'].min())
days_max = max(call_df['Days Till Expiration'].max(), put_df['Days Till Expiration'].max())
iv_min = min(call_df['impliedVolatility'].min(), put_df['impliedVolatility'].min())
iv_max = max(call_df['impliedVolatility'].max(), put_df['impliedVolatility'].max())

# Create vertical plane at spot price for both plots
days_grid = np.linspace(days_min, days_max, 30)
iv_grid = np.linspace(iv_min, iv_max, 30)
days_mesh, iv_mesh = np.meshgrid(days_grid, iv_grid)
strike_plane = np.full_like(days_mesh, spot_price)

# Add vertical plane to Calls subplot
fig.add_trace(go.Surface(
    x=strike_plane,
    y=days_mesh,
    z=iv_mesh,
    showscale=False,
    opacity=0.25,
    colorscale=[[0, 'gray'], [1, 'gray']],
    hoverinfo='skip',
    name='Spot Price Plane'
), row=1, col=1)

# Add vertical plane to Puts subplot
fig.add_trace(go.Surface(
    x=strike_plane,
    y=days_mesh,
    z=iv_mesh,
    showscale=False,
    opacity=0.25,
    colorscale=[[0, 'gray'], [1, 'gray']],
    hoverinfo='skip',
    name='Spot Price Plane'
), row=1, col=2)

# Update layout
fig.update_layout(
    height=800,
    title_text=f'{ticker_str} Call and Put Options: Raw IV & SVI Surfaces with Spot Price Plane',
    scene=dict(
        xaxis=dict(title='Strike', range=[strike_min, strike_max]),
        yaxis=dict(title='Days to Expiration', range=[days_min, days_max]),
        zaxis=dict(title='Implied Volatility', range=[iv_min, iv_max]),
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
    ),
    scene2=dict(
        xaxis=dict(title='Strike', range=[strike_min, strike_max]),
        yaxis=dict(title='Days to Expiration', range=[days_min, days_max]),
        zaxis=dict(title='Implied Volatility', range=[iv_min, iv_max]),
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
    )
)

all_dtes = np.unique(np.concatenate([
    call_df['Days Till Expiration'].unique(),
    put_df['Days Till Expiration'].unique()
]))

# Get RV surface
rv_df = get_realized_vol_surface(ticker_str, all_dtes)

# Create meshgrid for surface overlay at all strikes
strike_range = np.linspace(strike_min, strike_max, 50)
dte_rv_grid, strike_rv_grid = np.meshgrid(rv_df['Days Till Expiration'], strike_range)
vol_rv_grid = np.tile(rv_df['Realized Vol'].values, (len(strike_range), 1))

# Add RV to Call subplot
fig.add_trace(go.Surface(
    x=strike_rv_grid,
    y=dte_rv_grid,
    z=vol_rv_grid,
    colorscale=[[0, 'orange'], [1, 'orange']],
    opacity=0.5,
    showscale=False,
    name='Realized Vol Surface',
    hoverinfo='skip'
), row=1, col=1)

# Add RV to Put subplot
fig.add_trace(go.Surface(
    x=strike_rv_grid,
    y=dte_rv_grid,
    z=vol_rv_grid,
    colorscale=[[0, 'orange'], [1, 'orange']],
    opacity=0.5,
    showscale=False,
    name='Realized Vol Surface',
    hoverinfo='skip'
), row=1, col=2)

fig.show()


Spot price for UNH: 335.67999267578125


In [15]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata

# Ensure all rows and columns are displayed
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

def calc_positive_deviation_otm(df, strike_grid, T_grid, iv_svi_surface, spot_price, strike_range=100, limit=100, option_type='call'):
    """
    Calculate OTM options where market IV exceeds SVI IV, with positive deviation and distance from spot.
    """
    # Prepare SVI points and values for interpolation
    points = []
    values = []
    for i, T in enumerate(T_grid):
        for j, K in enumerate(strike_grid):
            points.append((K, T))
            values.append(iv_svi_surface[i, j])
    points = np.array(points)
    values = np.array(values)

    # Market points for interpolation
    market_points = np.vstack([df['strike'].values, df['T'].values]).T

    # Interpolate SVI IV at market points
    svi_iv_at_market = griddata(points, values, market_points, method='linear')

    df = df.copy()
    df['svi_iv'] = svi_iv_at_market

    # Filter by strike distance from spot price
    df = df[(df['strike'] >= spot_price - strike_range) & (df['strike'] <= spot_price + strike_range)]

    # Filter for OTM contracts
    if option_type.lower() == 'call':
        df = df[df['strike'] > spot_price]  # calls OTM
    elif option_type.lower() == 'put':
        df = df[df['strike'] < spot_price]  # puts OTM

    # Filter where market IV > SVI IV
    df_filtered = df[df['impliedVolatility'] > df['svi_iv']].copy()

    # Calculate positive deviation
    df_filtered['positive_deviation'] = df_filtered['impliedVolatility'] - df_filtered['svi_iv']

    # Add distance from spot column
    df_filtered['distance_from_spot'] = df_filtered['strike'] - spot_price

    # Sort descending by positive deviation
    df_filtered_sorted = df_filtered.sort_values(by='positive_deviation', ascending=False)

    # Select relevant columns and limit to top N
    result_df = df_filtered_sorted[['strike', 'Days Till Expiration', 'impliedVolatility', 'svi_iv', 'positive_deviation', 'distance_from_spot']].head(limit)

    return result_df.reset_index(drop=True)


# --- Example Usage ---
calls_deviations_otm = calc_positive_deviation_otm(
    df=call_df,
    strike_grid=call_strike_grid,
    T_grid=call_T_grid,
    iv_svi_surface=call_iv_svi_surface,
    spot_price=spot_price,
    strike_range=100,
    limit=100,
    option_type='call'
)

puts_deviations_otm = calc_positive_deviation_otm(
    df=put_df,
    strike_grid=put_strike_grid,
    T_grid=put_T_grid,
    iv_svi_surface=put_iv_svi_surface,
    spot_price=spot_price,
    strike_range=100,
    limit=100,
    option_type='put'
)

print("Top 100 OTM Calls with Market IV > SVI IV (within ±100 strikes):")
display(calls_deviations_otm)

print("\nTop 100 OTM Puts with Market IV > SVI IV (within ±100 strikes):")
display(puts_deviations_otm)


Top 100 OTM Calls with Market IV > SVI IV (within ±100 strikes):


,strike,Days Till Expiration,impliedVolatility,svi_iv,positive_deviation,distance_from_spot
0,340.0000,41,0.4474,0.3401,0.1073,4.3200
1,345.0000,41,0.4437,0.3410,0.1028,9.3200
2,350.0000,41,0.4459,0.3443,0.1016,14.3200
3,355.0000,41,0.4453,0.3486,0.0968,19.3200
4,360.0000,41,0.4467,0.3521,0.0946,24.3200
5,385.0000,34,0.4609,0.3694,0.0916,49.3200
6,375.0000,41,0.4541,0.3679,0.0862,39.3200
7,435.0000,34,0.5148,0.4321,0.0827,99.3200
8,365.0000,41,0.4382,0.3568,0.0814,29.3200
9,370.0000,41,0.4405,0.3626,0.0779,34.3200



Top 100 OTM Puts with Market IV > SVI IV (within ±100 strikes):


,strike,Days Till Expiration,impliedVolatility,svi_iv,positive_deviation,distance_from_spot
0,240.0000,62,0.4950,0.3327,0.1623,-95.6800
1,250.0000,62,0.4579,0.3329,0.1250,-85.6800
2,260.0000,62,0.4302,0.3386,0.0915,-75.6800
3,270.0000,62,0.4124,0.3342,0.0782,-65.6800
4,320.0000,237,0.3620,0.2911,0.0710,-15.6800
5,240.0000,-1,1.9688,1.9009,0.0679,-95.6800
6,330.0000,237,0.3517,0.2841,0.0676,-5.6800
7,310.0000,237,0.3588,0.3031,0.0557,-25.6800
8,300.0000,237,0.3611,0.3137,0.0474,-35.6800
9,280.0000,62,0.3908,0.3438,0.0470,-55.6800


In [16]:

#retreive the contracts for the first expiration date and display them as a dataframe
contracts = yf.Ticker(ticker_str).option_chain(options_expiration_dates['Expiration Date'][0])
call_contract_chain = pd.DataFrame(contracts.calls)
put_call_contract_chain = pd.DataFrame(contracts.puts)
underlying_data = contracts.underlying


columns_to_drop = ['currency', 'contractSize', 'percentChange', 'change']

call_contract_chain = call_contract_chain.drop(columns=columns_to_drop)
put_call_contract_chain = put_call_contract_chain.drop(columns=columns_to_drop)



In [17]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Get the current stock price from underlying data
current_price = underlying_data['regularMarketPrice']

# Create a function to filter and display options based on strike range
def show_options_table(strikes_to_show):
    # Calculate the strike range
    half_range = strikes_to_show // 2
    min_strike = current_price - half_range
    max_strike = current_price + half_range
    
    # Filter call options
    filtered_calls = call_contract_chain[
        (call_contract_chain['strike'] >= min_strike) & 
        (call_contract_chain['strike'] <= max_strike)
    ]
    
    # Filter put options
    filtered_puts = put_call_contract_chain[
        (put_call_contract_chain['strike'] >= min_strike) & 
        (put_call_contract_chain['strike'] <= max_strike)
    ]
    
    # Create a figure with two side-by-side tables
    fig = make_subplots(
        rows=1, cols=2,
        column_widths=[0.5, 0.5],
        subplot_titles=("Call Options", "Put Options"),
        specs=[[{"type": "table"}, {"type": "table"}]]
    )

    # Get expiration date from the options DataFrame
    expiration_date = options['Expiration Date'][0]
    
    # Create conditional fill colors for calls based on inTheMoney status
    call_fill_colors = []
    for itm in filtered_calls['inTheMoney']:
        if itm:
            call_fill_colors.append('rgba(144, 238, 144, 0.6)')  # Light green for in-the-money
        else:
            call_fill_colors.append('lavender')  # Default for out-of-the-money
    
    # Add call options table with conditional formatting
    fig.add_trace(
        go.Table(
            header=dict(values=list(filtered_calls.columns),
                        fill_color='paleturquoise',
                        align='left',
                        font=dict(size=11)),
            cells=dict(values=[filtered_calls[col] for col in filtered_calls.columns],
                       fill_color=[call_fill_colors for _ in filtered_calls.columns],
                       align='left')
        ),
        row=1, col=1
    )
    
    # Create conditional fill colors for puts based on inTheMoney status
    put_fill_colors = []
    for itm in filtered_puts['inTheMoney']:
        if itm:
            put_fill_colors.append('rgba(144, 238, 144, 0.6)')  # Light green for in-the-money
        else:
            put_fill_colors.append('mistyrose')  # Default for out-of-the-money
    
    # Add put options table with conditional formatting
    fig.add_trace(
        go.Table(
            header=dict(values=list(filtered_puts.columns),
                        fill_color='lightpink',
                        align='left'),
            cells=dict(values=[filtered_puts[col] for col in filtered_puts.columns],
                       fill_color=[put_fill_colors for _ in filtered_puts.columns],
                       align='left')
        ),
        row=1, col=2
    )
    
    # Update layout with title including the range information
    fig.update_layout(
        title_text=f"Option Chain for {ticker_str} - Expiration: {expiration_date}<br>" + 
                  f"<span style='font-size:12px;'>Green = In The Money | " +
                  f"Showing strikes from ${min_strike:.1f} to ${max_strike:.1f}</span>",
        height=800,
        autosize=True,
    )
    
    # Display with config that enables responsiveness
    fig.show(config={'responsive': True, 'scrollZoom': True})

# Create dropdown for strike selection
strike_dropdown = widgets.Dropdown(
    options=[(f"{i} Strikes", i) for i in [10, 20, 30, 40, 50, 60]],
    value=20,
    description='Show:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

# Function to handle dropdown change
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output(wait=True)
        display(strike_dropdown)
        show_options_table(change['new'])

# Register the callback
strike_dropdown.observe(on_change, names='value')

# Initial display
display(strike_dropdown)
show_options_table(strike_dropdown.value)


Dropdown(description='Show:', index=1, layout=Layout(width='200px'), options=(('10 Strikes', 10), ('20 Strikes…

NameError: name 'options' is not defined

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Define the ticker symbol
ticker_symbol = 'AAPL'

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Get the current stock price
current_price = ticker.history(period='1d')['Close'].iloc[-1]

# Get the expiration dates
try:
    expirations = ticker.options
    print("Expiration Dates:", expirations)
    
    # Choose an expiration date
    expiration_date = expirations[0]  # Modify as needed

    # Get options data for the selected expiration date
    options_data = ticker.option_chain(expiration_date)

    # Extract call and put options data
    calls = options_data.calls
    puts = options_data.puts

    # Convert calls and puts to dataframes
    calls_df = pd.DataFrame(calls)
    puts_df = pd.DataFrame(puts)

    # Function to decompose the contract symbol
    def decompose_contract_symbol(contract_symbol):
        try:
            ticker = contract_symbol[:4]
            expiration = f"20{contract_symbol[4:6]}-{contract_symbol[6:8]}-{contract_symbol[8:10]}"
            option_type = 'Call' if contract_symbol[10] == 'C' else 'Put'
            strike_price = int(contract_symbol[11:]) / 1000
            expiration_date = datetime.strptime(expiration, '%Y-%m-%d')
            return ticker, expiration, option_type, strike_price, expiration_date
        except Exception as e:
            print(f"Error decomposing symbol {contract_symbol}: {e}")
            return pd.Series([None, None, None, None, None])

    # Apply the function to the contract symbols
    calls_df[['Ticker', 'Expiration', 'OptionType', 'StrikePrice', 'ExpirationDate']] = calls_df['contractSymbol'].apply(
        lambda x: pd.Series(decompose_contract_symbol(x))
    )
    puts_df[['Ticker', 'Expiration', 'OptionType', 'StrikePrice', 'ExpirationDate']] = puts_df['contractSymbol'].apply(
        lambda x: pd.Series(decompose_contract_symbol(x))
    )

    # Calculate the number of days until expiration
    current_date = datetime.now()
    calls_df['DaysUntilExpiration'] = (calls_df['ExpirationDate'] - current_date).dt.days
    puts_df['DaysUntilExpiration'] = (puts_df['ExpirationDate'] - current_date).dt.days

    # Plot implied volatility across strike prices
    plt.figure(figsize=(12, 6))
    plt.plot(calls_df['StrikePrice'], calls_df['impliedVolatility'], label='Calls', marker='o')
    plt.plot(puts_df['StrikePrice'], puts_df['impliedVolatility'], label='Puts', marker='x')
    plt.axvline(x=current_price, color='r', linestyle='--', label='Current Price')
    plt.xlabel('Strike Price')
    plt.ylabel('Implied Volatility')
    plt.title(f'Implied Volatility across Strike Prices for {ticker_symbol} Options (Expiration: {expiration_date})')
    plt.legend()
    plt.grid(True)
    plt.show()

except Exception as e:
    print(f"Error fetching or processing data: {e}")